In [1]:
# ====================Importing All the libraries ====================
import pandas as pd
import numpy as np
import copy
import os
import plotly.graph_objects as go
from datetime import datetime

## Banking Files

In [2]:
#View all the files present in the folder
os.chdir('C:\\Users\\Manav\\Desktop\\Desktop files\\BYOP\\Banking data')

file_index=1
for folder,sub,files in os.walk(os.getcwd()):
    for file in files:
        print(f"{file_index}.",file)
        file_index+=1

1. Allahabad Bank.xlsx
2. Andhra Bank.xlsx
3. AXIS BANK.xlsx
4. Bank of Baroda.xlsx
5. BANK OF MAHARASHTRA.xlsx
6. BankNifty.csv
7. BOI.xlsx
8. Canara Bank.xlsx
9. Central Bank.xlsx
10. Corparation Bank.xlsx
11. Federal Bank.xlsx
12. HDFC Bank.xlsx
13. ICICI Bank.xlsx
14. IDBI.xlsx
15. Indian Bank.xlsx
16. Karnataka Bank.xlsx
17. Kotak Mahindra Bank.xlsx
18. Lakshmi Vilas Bank.xlsx
19. Oriental Bank.xlsx
20. PNB.xlsx
21. SBI.xlsx
22. SOUTH BANK.xlsx
23. Syndicate bank.xlsx
24. UCO Bank.xlsx
25. Union Bank.xlsx
26. YES BANK.xlsx


## Indicator ADX, Stochastic Oscillator

In [28]:
def Indicator(indata,n):
    #Adx indicator 
    indata['HL'] = indata['High Price'] - indata['Low Price']             
    indata['absHC'] = abs(indata['High Price'] - indata['Close Price'].shift(1))
    indata['absLC'] = abs(indata['Low Price'] - indata['Close Price'].shift(1))
    indata['TR'] = indata[['HL','absHC','absLC']].max(axis=1)
    indata['ATR'] = indata['TR'].rolling(window=14).mean()
    #indata['NATR'] = (indata['ATR'] / indata['Close Price']) *100
    indata = indata.fillna(0)
    
    indata['Up'] = indata['High Price'] - indata['High Price'].shift(1)
    indata['Down'] = indata['Low Price'].shift(1) - indata[' Low Price'] 
    indata['Zero'] = 0
    indata['+DM'] = np.where((indata['Up'] > indata['Down']) & (indata['Up'] > indata['Zero']), indata['Up'], 0) 
    indata['-DM'] = np.where((indata['Up'] < indata['Down']) & (indata['Down'] > indata['Zero']), indata['Down'], 0)
    indata['+DI'] = 100 * (indata['+DM']/indata['ATR']).ewm(span=14,min_periods=0,adjust=True,ignore_na=False).mean() 
    indata['-DI'] = 100 * (indata['-DM']/indata['ATR']).ewm(span=14,min_periods=0,adjust=True,ignore_na=False).mean()
    indata['ADX'] = 100 * (abs((indata['+DI'] - indata['-DI'])/(indata['+DI'] + indata['-DI']))).ewm(span=14,min_periods=0,adjust=True,ignore_na=False).mean()
    
    for stock in range(len(indata)):
        if (((indata.loc[stock,'ADX'])>=25) and (indata.loc[stock,'+DI'] >= indata.loc[stock,'-DI'])):
            indata.loc[stock,'Trend Signal'] = 'Uptrend'
        elif (((indata.loc[stock,'ADX'])>=25) and (indata.loc[stock,'+DI'] <= indata.loc[stock,'-DI'])):
            indata.loc[stock,'Trend Signal'] = 'Short Trade'
        else:
            indata.loc[stock,'Trend Signal'] = 'Trendless'
                   
    #Stochastic Indicator
    indata['SOK'] = abs(((indata['Close Price'] - indata['Low Price'].rolling(window=n, center=False).mean()) / (indata['High Price'].rolling(window=n, center=False).max() - indata['Low Price'].rolling(window=n, center=False).min())) * 100)
    indata['SOD'] = indata['SOK'].rolling(window = 3, center=False).mean()
        
    for i in range(len(indata)):
        if (indata.loc[i,'Trend Signal'] == 'Uptrend') & (indata.loc[i,'SOK'] >= 15) & (indata.loc[i,'SOK']<= 27):
            indata.loc[i,'Signal'] = 'Buy'
        elif (indata.loc[i,'Trend Signal'] == 'Short Trade') & (indata.loc[i,'SOK'] >= 75) & (indata.loc[i,'SOK']<= 100):
            indata.loc[i,'Signal'] = 'Sell'
        else:
            indata.loc[i,'Signal'] = 'NA'
         
    return indata

### Investment

In [29]:
file = input("Enter the file : ")
invest = 1000000

Enter the file : IDBI


In [11]:
start_date = input("Start Date (YYYY-MM-DD): ")

Start Date (YYYY-MM-DD): 2005-01-03


In [6]:
end_date = input("End Date (YYYY-MM-DD): ")

End Date (YYYY-MM-DD): 2005-06-30


In [30]:
data = pd.read_excel(f"{file}.xlsx")
indata = copy.deepcopy(data)
indata['Date']=pd.to_datetime(indata['Date'])
indata=indata.drop(columns=['Total Traded Quantity', 'Turnover', 'No. of Trades', 'Deliverable Qty','Series','% Dly Qt to Traded Qty','Average Price'])
print(indata.head())

Benchmark = pd.read_csv("Banknifty.csv")
Benchmark['Date']=pd.to_datetime(Benchmark['Date'])
Benchmark.head()

  Symbol       Date  Prev Close  Open Price  High Price  Low Price  \
0   IDBI 2000-01-03       36.90       39.85       39.85      39.85   
1   IDBI 2000-01-04       39.85       43.05       43.05      38.00   
2   IDBI 2000-01-05       42.10       41.00       43.00      39.00   
3   IDBI 2000-01-06       39.45       39.10       41.50      37.70   
4   IDBI 2000-01-07       38.65       39.20       39.20      37.50   

   Last Price  Close Price  
0       39.85        39.85  
1       42.90        42.10  
2       39.95        39.45  
3       38.70        38.65  
4       38.00        37.80  


,Date,Open Price,High Price,Low Price,Close Price,Volume,Turnover
0,2000-01-01,NaN,NaN,NaN,1000.00,NaN,NaN
1,2000-03-01,NaN,NaN,NaN,1054.81,NaN,NaN
2,2000-04-01,NaN,NaN,NaN,1092.08,NaN,NaN
3,2000-05-01,NaN,NaN,NaN,1055.99,NaN,NaN
4,2000-06-01,NaN,NaN,NaN,1086.67,NaN,NaN


In [31]:
indata = Indicator(indata, 14)

In [32]:
indata.head(50)

,Symbol,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,HL,absHC,...,Zero,+DM,-DM,+DI,-DI,ADX,Trend Signal,SOK,SOD,Signal
0,IDBI,2000-01-03,36.90,39.85,39.85,39.85,39.85,39.85,0.00,0.00,...,0,0.00,0.00,NaN,NaN,NaN,Trendless,NaN,NaN,NA
1,IDBI,2000-01-04,39.85,43.05,43.05,38.00,42.90,42.10,5.05,3.20,...,0,3.20,0.00,NaN,NaN,NaN,Trendless,NaN,NaN,NA
2,IDBI,2000-01-05,42.10,41.00,43.00,39.00,39.95,39.45,4.00,0.90,...,0,0.00,0.00,NaN,NaN,NaN,Trendless,NaN,NaN,NA
3,IDBI,2000-01-06,39.45,39.10,41.50,37.70,38.70,38.65,3.80,2.05,...,0,0.00,1.30,NaN,NaN,NaN,Trendless,NaN,NaN,NA
4,IDBI,2000-01-07,38.65,39.20,39.20,37.50,38.00,37.80,1.70,0.55,...,0,0.00,0.20,NaN,NaN,NaN,Trendless,NaN,NaN,NA
5,IDBI,2000-01-10,37.80,38.50,38.55,37.55,37.75,37.75,1.00,0.75,...,0,0.00,0.00,NaN,NaN,NaN,Trendless,NaN,NaN,NA
6,IDBI,2000-01-11,37.75,38.40,38.75,36.00,36.25,36.25,2.75,1.00,...,0,0.00,1.55,NaN,NaN,NaN,Trendless,NaN,NaN,NA
7,IDBI,2000-01-12,36.25,37.95,37.95,36.90,37.45,37.40,1.05,1.70,...,0,0.00,0.00,NaN,NaN,NaN,Trendless,NaN,NaN,NA
8,IDBI,2000-01-13,37.40,37.95,38.00,36.70,36.95,36.85,1.30,0.60,...,0,0.00,0.20,NaN,NaN,NaN,Trendless,NaN,NaN,NA
9,IDBI,2000-01-14,36.85,37.75,37.75,36.55,37.00,36.80,1.20,0.90,...,0,0.00,0.15,NaN,NaN,NaN,Trendless,NaN,NaN,NA


### Return Analysis

In [33]:
#On Stock Investment
Stock_Investment = indata[(indata['Date']>=start_date) & (indata['Date'] <= end_date)]
s_invest = float(indata[indata['Date']==start_date]['Close Price'])
shares = int(invest/s_invest)
e_invest = float(indata[indata['Date']==end_date]['Close Price'])
Stock_Return = (shares * e_invest)

# BenchMark Investment
BM_Investment = Benchmark[(Benchmark['Date']>=start_date) & (Benchmark['Date'] <= end_date)]
Bs_invest = float(Benchmark[Benchmark['Date']==start_date]['Close Price'])
Bshares = int(invest/Bs_invest)
Be_invest = float(Benchmark[Benchmark['Date']==end_date]['Close Price'])
BM_Return = (Bshares * Be_invest)

print("============STOCK RETURN============")
print("The Close price on Start date : Rs",s_invest)
print("No. of shares invested :",shares)
print("The Close price on End date : Rs",e_invest)
print("The Return of your investment : Rs",Stock_Return)
print("\n")
print("==========BENCHMARK RETURN===========")
print("The Close price on Start date : Rs",Bs_invest)
print("No. of shares invested :",Bshares)
print("The Close price on End date : Rs",Be_invest)
print("The Return of your investment : Rs",BM_Return)

============STOCK RETURN============
The Close price on Start date : Rs 113.05
No. of shares invested : 8845
The Close price on End date : Rs 101.7
The Return of your investment : Rs 899536.5


==========BENCHMARK RETURN===========
The Close price on Start date : Rs 3627.06
No. of shares invested : 275
The Close price on End date : Rs 3638.4
The Return of your investment : Rs 1000560.0


In [26]:
Signal = indata[['Date', 'Prev Close', 'Open Price', 'High Price', 'Low Price',
                   'Last Price', 'Close Price','+DI', '-DI', 'ADX','Trend Signal', 'SOK', 'SOD', 'Signal']]
Signal

,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,+DI,-DI,ADX,Trend Signal,SOK,SOD,Signal
0,2000-01-03,225.60,236.00,243.65,234.25,243.65,243.65,NaN,NaN,NaN,Trendless,NaN,NaN,NA
1,2000-01-04,243.65,243.65,262.00,238.85,258.00,259.10,NaN,NaN,NaN,Trendless,NaN,NaN,NA
2,2000-01-05,259.10,249.00,264.70,245.00,249.05,248.45,NaN,NaN,NaN,Trendless,NaN,NaN,NA
3,2000-01-06,248.45,252.00,268.00,252.00,260.50,261.00,NaN,NaN,NaN,Trendless,NaN,NaN,NA
4,2000-01-07,261.00,261.50,279.90,255.00,279.00,273.30,NaN,NaN,NaN,Trendless,NaN,NaN,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5224,2020-12-24,263.00,265.00,268.85,264.65,266.85,266.85,26.658025,27.273439,36.988664,Short Trade,10.867347,9.564445,NA
5225,2020-12-28,266.85,268.75,276.00,268.00,275.75,275.20,35.478834,23.636981,34.727724,Uptrend,39.413265,16.777211,NA
5226,2020-12-29,275.20,276.30,279.90,274.70,277.75,277.90,37.708170,20.485383,34.043454,Uptrend,41.266080,30.515564,NA
5227,2020-12-30,277.90,278.00,278.70,272.50,275.50,276.90,32.680414,21.661389,32.207960,Uptrend,37.361769,39.347038,NA


### Plot the charts 

In [27]:
chart = Signal[(Signal['Date']>= start_date) & (Signal['Date']<= end_date)]
chart1 = chart[['Date','+DI','-DI', 'ADX']]
chart2 = chart[['Date','SOK','SOD']]
import plotly.express as px
fig = go.Figure(data=[go.Candlestick(x=Stock_Investment['Date'],
                open=Stock_Investment['Open Price'],
                high=Stock_Investment['High Price'],
                low=Stock_Investment['Low Price'],
                close=Stock_Investment['Close Price'])])

fig.show()
#ADX_Indicator = px.data.stocks()
fig1 = px.line(chart1, x="Date", y = chart1.columns,
              hover_data={"Date": "|%B %d, %Y"})
              #title='custom tick labels')
fig1.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig1.show()

fig2 = px.line(chart2, x="Date", y = chart2.columns,
              hover_data={"Date": "|%B %d, %Y"})
              #title='custom tick labels')
fig2.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig2.show()